In [1]:
# -*- coding: utf8 -*-
import codecs
import features
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools as it
import pickle

In [2]:
import datetime
def dateheure():
    return datetime.datetime.utcnow().strftime('%y%m%d%H%M')

typeEchantillon : 
- fixe pour utiliser tailleEchantillon
- variable pour utiliser nombre,increment
- total pour utiliser tout le lexique

In [3]:
typeEchantillon="variable"
tailleEchantillon=15000
nombre=5
nombreEchantillons=20
increment=15000
freqForme=1000000
if typeEchantillon=="total":
    echantillonPrefix="-total-%s" % (dateheure())
elif typeEchantillon=="fixe":
    echantillonPrefix="-%d-%s" % (tailleEchantillon,dateheure())
else:
    echantillonPrefix="-extend"

In [4]:
filePrefix="MGC-151206"
lexiquePrefix="MGC-150815"

##Importations
- codecs pour les encodages
- pandas et numpy pour les calculs sur tableaux
- matplotlib pour les graphiques
- itertools pour les itérateurs sophistiqués (paires sur liste, ...)

In [5]:
#%pylab inline
#pd.options.display.mpl_style = 'default'
debug=False

In [6]:
bdlexiqueIn = unicode(u"èò")
bdlexiqueNum = [ord(char) for char in bdlexiqueIn]
neutreOut = unicode(u"EO")
neutralise = dict(zip(bdlexiqueNum, neutreOut))

In [7]:
def recoder(chaine,table=neutralise):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp.encode('utf8')
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    return result

###Définition des cases
- principales pour le paradigme courant
- secondaires pour les cases rares
- totales pour le tout

In [8]:
casesPrincipales= [
        'inf', 'pi1S', 'pi2S', 'pi3S', 'pi1P', 'pi2P', 'pi3P', 'ii1S',
        'ii2S', 'ii3S', 'ii1P', 'ii2P', 'ii3P', 
        'fi1S', 'fi2S', 'fi3S', 'fi1P', 'fi2P',
        'fi3P', 'pI2S', 'pI1P', 'pI2P', 'ps1S', 'ps2S', 'ps3S', 'ps1P',
        'ps2P', 'ps3P', 
        'pc1S', 'pc2S', 'pc3S', 'pc1P', 'pc2P', 'pc3P', 'pP',
        'ppMS', 'ppMP', 'ppFS', 'ppFP'
            ]
casesSecondaires= [
       'ai1S', 'ai2S', 'ai3S', 'ai1P', 'ai2P', 'ai3P', 'is1S', 'is2S', 'is3S', 'is1P', 'is2P', 'is3P'
            ]
casesTotales=casesPrincipales+casesSecondaires
listeCases=casesTotales

In [9]:
with open("2015-Data/"+lexiquePrefix+'-Verbes.pkl', 'rb') as input:
    verbes = pickle.load(input)

In [10]:
verbes["phono"]=verbes["phono"].apply(lambda x: recoder(x))

#Echantillonage

##Assembler les échantillons correspondant à une étape n

In [11]:
def assemblerExtrait(nombre):
    return verbes.ix[np.sort(np.concatenate(tirages[0:nombre]))]

###Tirage incrémental des formes disponibles
- increment : taille de chaque tirage
- nombre : nombre de tirages

np.random.choice donne une liste d'index de formes tirées dans l'ordre du tirage

on découpe la liste en morceaux de la taille de l'incrément

In [13]:
if typeEchantillon=="total":
    extrait=verbes
    paradigmes=pd.pivot_table(extrait, values='phono', index=['lexeme'], columns=['case'], aggfunc=lambda x: ",".join(x)).reset_index().reindex()
    paradigmes.to_csv(path_or_buf="2015-Data/"+filePrefix+echantillonPrefix+"-paradigmes.csv",encoding="utf8",sep=";")
else:
    for numEchantillon in range(nombreEchantillons):
        if typeEchantillon=="variable":
            tirage=np.random.choice(verbes.index,size=nombre*increment,p=verbes["prob"],replace=False)
            tirages=[tirage[increment*x:increment*(x+1)] for x in range(len(tirage)/increment+(len(tirage)%increment!=0))]
            for i in range(nombre):
                n=i+1
                extrait=assemblerExtrait(n)
                paradigmes=pd.pivot_table(extrait, values='phono', index=['lexeme'], columns=['case'], aggfunc=lambda x: ",".join(x)).reset_index().reindex()
                paradigmes.to_csv(path_or_buf="2015-Data/"+filePrefix+echantillonPrefix+"-%02d"%numEchantillon+"-%02d"%n+"-paradigmes.csv",encoding="utf8",sep=";")
        else:
            if typeEchantillon=="fixe":
                tirage=np.random.choice(verbes.index,size=tailleEchantillon,p=verbes["prob"],replace=False)
                extrait=verbes.ix[np.sort(tirage)]
            paradigmes=pd.pivot_table(extrait, values='phono', index=['lexeme'], columns=['case'], aggfunc=lambda x: ",".join(x)).reset_index().reindex()
            paradigmes.to_csv(path_or_buf="2015-Data/"+filePrefix+echantillonPrefix+"-%02d"%numEchantillon+"-paradigmes.csv",encoding="utf8",sep=";")

In [ ]:
#verbes.ix[tirages[0]].sort(columns=["prob"],ascending=False)

In [ ]:
#for i in range(nombre):
#    print verbes.ix[tirages[i]]["prob"].max(),"\t", verbes.ix[tirages[i]]["prob"].median()